***GENERATED CODE FOR eligdenresolutionrecommendationautoml PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '765', 'mean': '85129.06', 'stddev': '25320.65', 'min': '11042', 'max': 'Q0138', 'missing': '0', 'distinct': '96'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis1', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'A04.72', 'max': 'Z95.4', 'missing': '0', 'distinct': '286'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis1'}, {'feature_label': 'pdiagnosis1', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis1')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis2', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'B35.1', 'max': 'Z99.2', 'missing': '0', 'distinct': '308'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis2'}, {'feature_label': 'pdiagnosis2', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis2')
        transformationDF = transformationDF.drop('Age')
        transformationDF = transformationDF.drop('Gender')
        transformationDF = transformationDF.drop('Amount')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'Cardiovascular', 'max': 'Urodyamics', 'missing': '0', 'distinct': '28'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SpecialityCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'Cardiac Surgery', 'max': 'Vascular Surgery', 'missing': '0', 'distinct': '23'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SpecialityCategory'}, {'feature_label': 'SpecialityCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SpecialityCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'Work Comp', 'missing': '0', 'distinct': '18'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerCategory'}, {'feature_label': 'PrimaryPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'Work Comp', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerGroup'}, {'feature_label': 'PrimaryPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'PrimaryPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Work Comp', 'missing': '0', 'distinct': '58'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PrimaryPayerName'}, {'feature_label': 'PrimaryPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PrimaryPayerName')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'Aetna US Healthcare', 'max': 'United Healthcare', 'missing': '0', 'distinct': '14'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerCategory'}, {'feature_label': 'SecondPayerCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerGroup', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'Aetna', 'max': 'UHC', 'missing': '0', 'distinct': '12'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerGroup'}, {'feature_label': 'SecondPayerGroup', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerGroup')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'}], 'feature': 'SecondPayerName', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'AARP 2ndary', 'max': 'Wellcare Staywell', 'missing': '0', 'distinct': '41'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'SecondPayerName'}, {'feature_label': 'SecondPayerName', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('SecondPayerName')
        transformationDF = transformationDF.drop('NoServiceLocation')
        transformationDF = transformationDF.drop('ReasonDay')
        transformationDF = transformationDF.drop('ReasonMonth')
        transformationDF = transformationDF.drop('Age_Bins')
        transformationDF = transformationDF.drop('Amount_Bins')
        transformationDF = transformationDF.drop('reasonType')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'}], 'feature': 'Reasoncode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '30.67', 'stddev': '24.16', 'min': '109', 'max': '31', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Reasoncode'}, {'feature_label': 'Reasoncode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Reasoncode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '765', 'mean': '', 'stddev': '', 'min': 'Benefits Max/Exh', 'max': 'Eligibility', 'missing': '0', 'distinct': '4'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonCategory'}, {'feature_label': 'ReasonCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'}], 'feature': 'ReasonDescription', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {'count': '765', 'mean': '', 'stddev': '', 'min': 'Charges are covered under a capitation agreement/managed care plan.',
                                                                                                                                                                                                                                                                                    'max': 'This is a work-related injury/illness and thus the liability of the Worker`s Compensation Carrier.', 'missing': '0', 'distinct': '10'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ReasonDescription'}, {'feature_label': 'ReasonDescription', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ReasonDescription')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'NextGen_BOND_CLINIC', 'optionalDB': 'NextGen_BOND_CLINIC', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT\r\n\r\ncd.ChargeDetailID,\r\ncptc.CPTCode,\r\ndc.DiagnosisCode as pdiagnosis1,\r\ndc2.DiagnosisCode as pdiagnosis2,\r\nFLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) AS Age, p.Gender,\r\ncd.Amount,\r\nd.DepartmentCategory,\r\ns.SpecialityCategory,\r\n\r\nCASE WHEN py.PayerCategory IS NOT NULL THEN py.PayerCategory ELSE 'NOT AVAILABLE' END as PrimaryPayerCategory ,\r\nCASE WHEN py.PayerGroup IS NOT NULL THEN py.PayerGroup ELSE 'NOT AVAILABLE' END as PrimaryPayerGroup ,\r\nCASE WHEN py.PayerName IS NOT NULL THEN py.PayerName ELSE 'NOT AVAILABLE' END as PrimaryPayerName ,\r\n\r\nCASE WHEN py1.PayerCategory IS NOT NULL THEN py1.PayerCategory ELSE 'NOT AVAILABLE' END as SecondPayerCategory ,\r\nCASE WHEN py1.PayerGroup IS NOT NULL THEN py1.PayerGroup ELSE 'NOT AVAILABLE' END as SecondPayerGroup ,\r\nCASE WHEN py1.PayerName IS NOT NULL THEN py1.PayerName ELSE 'NOT AVAILABLE' END as SecondPayerName ,\r\n\r\nCASE WHEN sl.ServiceLocation IS NULL THEN 1 ELSE 0 END as NoServiceLocation,\r\nDATEPART(WEEKDAY, tr.ReasonDate) as ReasonDay, \r\nDATEPART(MONTH, tr.ReasonDate) as ReasonMonth, \r\n\r\n\t\t\t\tCASE WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) >= 0 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=30 THEN '<=30' \r\n\t\t\t\t\t WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 30 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=50 THEN '31-50'\r\n\t\t\t\t\t WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 50 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=70 THEN '51-70'\r\n\t\t\t\t\t WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 70 and FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) <=90 THEN '71-90'\r\n\t\t\t\t\t WHEN FLOOR(DATEDIFF(DAY, p.DateOfBirth, GETDATE()) / 365) > 90 THEN '90+' END AS Age_Bins, \r\n\r\n\t\t\t\tCASE WHEN cd.Amount >= 0 and cd.Amount <=100 THEN '<=100' \r\n\t\t\t\t\t WHEN cd.Amount > 100 and cd.Amount <=300 THEN '101-300'\r\n\t\t\t\t\t WHEN cd.Amount > 300 and cd.Amount <=500 THEN '301-500'\r\n\t\t\t\t\t WHEN cd.Amount > 500 and cd.Amount <=700 THEN '501-700'\r\n\t\t\t\t\t WHEN cd.Amount > 700 and cd.Amount <=1000 THEN '701-1000' \r\n\t\t\t\t\t WHEN cd.Amount > 1000 THEN '1000+' END AS Amount_Bins, \r\ntrc.reasonType,\r\ntrc.Reasoncode,\r\ntrc.ReasonCategory,\r\ntrc.ReasonDescription,\r\n\r\nCASE\r\n\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 6 THEN 6\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 8 THEN 8\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 13 THEN 13\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 16 THEN 16\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 41 THEN 41\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 42 THEN 42\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 43 THEN 43\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 44 THEN 44\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 48 THEN 48\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 49 THEN 49\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 61 THEN 61\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 62 THEN 62\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 63 THEN 63\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 64 THEN 64\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 65 THEN 65\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 79 THEN 79\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 80 THEN 80\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 81 THEN 81\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 82 THEN 82\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 83 THEN 83\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 85 THEN 85\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 89 THEN 89\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 789 THEN 789\r\nWHEN t.transactiontypeid IN ('1','2') and t.Amount >= 0 and tcr.TaskCompletionReasonID = 30899 THEN 30899\r\n\r\nELSE '0' END AS label_resolution\r\n\r\nFROM\r\nChargeDetail cd\r\ninner join Transactions t WITH (NOLOCK) on cd.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionReason tr WITH (NOLOCK) on tr.ChargeDetailID = t.ChargeDetailID AND T.TransactionDate>=TR.ReasonDate\r\ninner join TransactionReasonCode trc WITH (NOLOCK) on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Payer py WITH (NOLOCK) on py.PayerID = t.PayerID\r\ninner join Transactiontype tt on tt.transactiontypeid = t.transactiontypeid\r\ninner join Encounter e WITH (NOLOCK) on cd.DateOfService = e.DateOfService and cd.PatientID = e.PatientID\r\nleft outer join AuthorizationsPA apa WITH (NOLOCK) on e.PatientID = apa.PatientID and apa.PayerId = t.PayerID\r\ninner join Bill b WITH (NOLOCK) on e.EncounterID = b.EncounterID and b.BillID = t.BillID\r\nleft outer join Task tsk WITH (NOLOCK) on e.encounterid = tsk.encounterid and e.facilityid = tsk.facilityid and tsk.createddate >= tr.reasondate\r\nleft outer join TaskCompletionReason tcr WITH (NOLOCK) on tsk.TaskCompletionReasonID = tcr.TaskCompletionReasonID\r\nleft outer join Taskstatus ts on tsk.TaskStatusID=ts.TaskStatusID\r\nleft outer join TaskType tskt on tsk.TaskTypeID=tskt.TaskTypeID\r\ninner join tmp_ResolutionRankings RR on tsk.TaskCompletionReasonID=RR.TaskCompletionReasonID\r\ninner join Patient p WITH (NOLOCK) on p.PatientID = cd.PatientID\r\ninner join Department d WITH (NOLOCK) on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp WITH (NOLOCK) on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s WITH (NOLOCK) on s.SpecialityID = pp.SpecialityID\r\ninner join DiagnosisCode dc WITH (NOLOCK) on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\ninner join DiagnosisCode dc2 WITH (NOLOCK) on dc2.DiagnosisCodeID = cd.DiagnosisCodeID2\r\ninner join CptCode cptc WITH (NOLOCK) on cptc.CPTCodeID = cd.CPTCodeID\r\nleft join Payer py1 WITH (NOLOCK) on py1.PayerID = cd.SecondaryPayerID\r\nleft join Payer py2 WITH (NOLOCK) on py2.PayerID = cd.TertiaryPayerID\r\ninner join ServiceItem si WITH (NOLOCK) on cd.ServiceItemID = si.ServiceItemID\r\nleft outer join ServiceLocation sl WITH (NOLOCK) on cd.ServiceLocationID = sl.ServiceLocationID\r\n\r\nWHERE cd.Amount >.01\r\nand trc.ReasonCode IN ('19','20','21','22','24','26','27','28','30','31','32','33','109','166','200')\r\nand trc.reasonType = 'Denial'\r\nand tr.ReasonDate between '2018-01-01' and '2020-12-31'\r\nand ts.TaskStatusCategory ='Completed'\r\nand tcr.TaskCompletionReasonID NOT IN ('99','100','101')\r\nand tcr.TaskCompletionReason !='NULL'\r\nand tsk.CreatedDate >= tr.ReasonDate\r\nand TransactionDate >= tsk.CompletedDate\r\nand tskt.TaskTypeCategory='AR'\r\nand RR.Rank = 1", 'dbtype': 'mssql', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "00436B8F-E8B1-4465-A272-0B201EE95B31", "max": "FFC1C276-FC2E-4DA6-872D-1FA5B9DE8DFA", "missing": "0", "distinct": "666"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "85129.06", "stddev": "25320.65", "min": "11042", "max": "Q0138", "missing": "0", "distinct": "96"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "pdiagnosis1", "transformation_label": "String Indexer"}], "feature": "pdiagnosis1", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "A04.72", "max": "Z95.4", "missing": "0", "distinct": "286"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis1"}, {"transformationsData": [{"feature_label": "pdiagnosis2", "transformation_label": "String Indexer"}], "feature": "pdiagnosis2", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "B35.1", "max": "Z99.2", "missing": "0", "distinct": "308"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis2"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Age", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "765", "mean": "71.6", "stddev": "16.58", "min": "2", "max": "103", "missing": "0"}, "updatedLabel": "Age"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0", "distinct": "2"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Amount", "type": "real", "selected": "False", "replaceby": "mean", "stats": {"count": "765", "mean": "170.61", "stddev": "361.15", "min": "0.56", "max": "8617.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Amount"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Cardiovascular", "max": "Urodyamics", "missing": "0", "distinct": "28"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "SpecialityCategory", "transformation_label": "String Indexer"}], "feature": "SpecialityCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Cardiac Surgery", "max": "Vascular Surgery", "missing": "0", "distinct": "23"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SpecialityCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerCategory", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "Work Comp", "missing": "0", "distinct": "18"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerCategory"}, {"transformationsData": [{"feature_label": "PrimaryPayerGroup", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Aetna", "max": "Work Comp", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerGroup"}, {"transformationsData": [{"feature_label": "PrimaryPayerName", "transformation_label": "String Indexer"}], "feature": "PrimaryPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Work Comp", "missing": "0", "distinct": "58"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PrimaryPayerName"}, {"transformationsData": [{"feature_label": "SecondPayerCategory", "transformation_label": "String Indexer"}], "feature": "SecondPayerCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Aetna US Healthcare", "max": "United Healthcare", "missing": "0", "distinct": "14"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerCategory"}, {"transformationsData": [{"feature_label": "SecondPayerGroup", "transformation_label": "String Indexer"}], "feature": "SecondPayerGroup", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Aetna", "max": "UHC", "missing": "0", "distinct": "12"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerGroup"}, {"transformationsData": [{"feature_label": "SecondPayerName", "transformation_label": "String Indexer"}], "feature": "SecondPayerName", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "AARP 2ndary", "max": "Wellcare Staywell", "missing": "0", "distinct": "41"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "SecondPayerName"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "NoServiceLocation", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "765", "mean": "0.93", "stddev": "0.25", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "NoServiceLocation"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ReasonDay", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "765", "mean": "3.85", "stddev": "1.51", "min": "1", "max": "7", "missing": "0"}, "updatedLabel": "ReasonDay"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "ReasonMonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "False", "stats": {"count": "765", "mean": "5.32", "stddev": "3.25", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "ReasonMonth"}, {"transformationsData": [{"feature_label": "Age_Bins", "transformation_label": "String Indexer"}], "feature": "Age_Bins", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Age_Bins"}, {"transformationsData": [{"feature_label": "Amount_Bins", "transformation_label": "String Indexer"}], "feature": "Amount_Bins", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Amount_Bins"}, {"transformationsData": [{"feature_label": "reasonType", "transformation_label": "String Indexer"}], "feature": "reasonType", "type": "string", "selected": "False", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Denial", "max": "Denial", "missing": "0", "distinct": "1"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "reasonType"}, {"transformationsData": [{"feature_label": "Reasoncode", "transformation_label": "String Indexer"}], "feature": "Reasoncode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "30.67", "stddev": "24.16", "min": "109", "max": "31", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Reasoncode"}, {"transformationsData": [{"feature_label": "ReasonCategory", "transformation_label": "String Indexer"}], "feature": "ReasonCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Benefits Max/Exh", "max": "Eligibility", "missing": "0", "distinct": "4"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonCategory"}, {"transformationsData": [{"feature_label": "ReasonDescription", "transformation_label": "String Indexer"}], "feature": "ReasonDescription", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "765", "mean": "", "stddev": "", "min": "Charges are covered under a capitation agreement/managed care plan.", "max": "This is a work-related injury/illness and thus the liability of the Worker`s Compensation Carrier.", "missing": "0", "distinct": "10"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ReasonDescription"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "label_resolution", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "765", "mean": "48.09", "stddev": "19.67", "min": "0", "max": "65", "missing": "0"}, "updatedLabel": "label_resolution"}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "10.93", "stddev": "19.45", "min": "0.0", "max": "95.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "pdiagnosis1_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "73.1", "stddev": "79.17", "min": "0.0", "max": "285.0", "missing": "0"}, "updatedLabel": "pdiagnosis1_stringindexer..."}, {"feature": "pdiagnosis2_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "81.45", "stddev": "86.07", "min": "0.0", "max": "307.0", "missing": "0"}, "updatedLabel": "pdiagnosis2_stringindexer..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "2.76", "stddev": "4.58", "min": "0.0", "max": "27.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "SpecialityCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "3.75", "stddev": "4.65", "min": "0.0", "max": "22.0", "missing": "0"}, "updatedLabel": "SpecialityCategory_string..."}, {"feature": "PrimaryPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "1.52", "stddev": "2.33", "min": "0.0", "max": "17.0", "missing": "0"}, "updatedLabel": "PrimaryPayerCategory_stri..."}, {"feature": "PrimaryPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "1.05", "stddev": "1.72", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "PrimaryPayerGroup_stringi..."}, {"feature": "PrimaryPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "7.36", "stddev": "11.05", "min": "0.0", "max": "57.0", "missing": "0"}, "updatedLabel": "PrimaryPayerName_stringin..."}, {"feature": "SecondPayerCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "1.32", "stddev": "2.1", "min": "0.0", "max": "13.0", "missing": "0"}, "updatedLabel": "SecondPayerCategory_strin..."}, {"feature": "SecondPayerGroup_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "1.18", "stddev": "1.78", "min": "0.0", "max": "11.0", "missing": "0"}, "updatedLabel": "SecondPayerGroup_stringin..."}, {"feature": "SecondPayerName_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "3.35", "stddev": "6.45", "min": "0.0", "max": "40.0", "missing": "0"}, "updatedLabel": "SecondPayerName_stringind..."}, {"feature": "Reasoncode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "1.04", "stddev": "1.52", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "Reasoncode_stringindexer_..."}, {"feature": "ReasonCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "765", "mean": "0.76", "stddev": "0.94", "min": "0", "max": "3", "missing": "0"}, "updatedLabel": "ReasonCategory_stringinde..."}, {"feature": "ReasonDescription_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "765", "mean": "1.04", "stddev": "1.52", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "ReasonDescription_stringi..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run eligdenresolutionrecommendationautomlHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["CPTCode_stringindexer", "pdiagnosis1_stringindexer", "pdiagnosis2_stringindexer", "DepartmentCategory_stringindexer", "SpecialityCategory_stringindexer", "PrimaryPayerCategory_stringindexer", "PrimaryPayerGroup_stringindexer", "PrimaryPayerName_stringindexer", "SecondPayerCategory_stringindexer", "SecondPayerGroup_stringindexer", "SecondPayerName_stringindexer", "Reasoncode_stringindexer", "ReasonCategory_stringindexer", "ReasonDescription_stringindexer"], "label_resolution")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

